In [12]:
# install Catboost
! pip install catboost

In [13]:
# Load libraries
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
import pandas as pd

In [14]:
# Load dataset and drop unneeded columns
df=pd.read_table('/content/drive/MyDrive/fungi_data.txt', on_bad_lines="skip", encoding='cp1252', delimiter="\t")
df=df[['plot','Last_fire_y_ago', 'biome', 'nativeness','warning', 'delta15N', 'N_conc_g_kg', 'P_conc_mg_kg', 'abund', 'richness' ]]
df

,plot,Last_fire_y_ago,biome,nativeness,warning,delta15N,N_conc_g_kg,P_conc_mg_kg,abund,richness
0,10MR,100,forest: temperate coniferous forest biome,NaN,NaN,-0.446175102,2.349947581,30.66579235,1718,380
1,11MR,100,forest: temperate coniferous forest biome,NaN,NaN,1.958914608,8.607632877,62.25348279,2908,669
2,12MR,100,forest: temperate coniferous forest biome,NaN,NaN,0.787812682,2.860775081,9.591139565,1842,514
3,13MR,100,forest: temperate coniferous forest biome,NaN,NaN,1.911578324,3.896421684,20.44708227,1010,370
4,14MR,100,forest: temperate coniferous forest biome,NaN,NaN,0.150616451,3.435929682,11.42307692,2440,518
...,...,...,...,...,...,...,...,...,...,...
3195,TR055,100,anthropogenic: cropland biome,NaN,NaN,3.017526686,3.488803847,41.91356431,3267,659
3196,TR060,100,anthropogenic: cropland biome,NaN,NaN,3.463565899,6.800029375,73.81886088,1660,574
3197,TR064,100,anthropogenic: cropland biome,NaN,NaN,4.338935739,1.938107722,106.1364112,3058,606
3198,TR065,100,anthropogenic: cropland biome,NaN,NaN,6.807058453,1.696994407,194.6655077,1547,425


In [15]:
# Fill missing values
df['nativeness']=df['nativeness'].fillna('native')
df['warning'] = df['warning'].fillna('no concern')
df['Last_fire_y_ago'].replace('unknown', '-999', inplace=True)
df['Last_fire_y_ago']=df['Last_fire_y_ago'].astype('float64')
df[['delta15N',	'N_conc_g_kg',	'P_conc_mg_kg']]=df[['delta15N',	'N_conc_g_kg',	'P_conc_mg_kg']].replace('unknown', '-999')
df[['delta15N',	'N_conc_g_kg',	'P_conc_mg_kg']]=df[['delta15N',	'N_conc_g_kg',	'P_conc_mg_kg']].astype('float64')

In [16]:
# Clean Biome and Biome Specific features
df[['Biome', 'Biome specific']]=df['biome'].str.split(':', n=1, expand=True)
df.drop('biome', axis=1, inplace=True)

In [17]:
# Label encode the features
import pickle
for i in ['nativeness', 'warning', 'Biome', 'Biome specific']:
  le=LabelEncoder()
  name= i +  '.pkl'
  output = open(name, 'wb')
  df[i]=le.fit_transform(df[i])
  pickle.dump(le, output)
  output.close()

In [18]:
df_cleaned=df.copy()

**Fungi Abundance**

In [19]:
# Split to dependent and independent variables
X=df.drop(['abund', 'richness', 'plot'], axis=1)
y= df['abund']
# split to train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.07, random_state=42)

In [20]:
# Build regressor model for fungi abundance prediction
rf=RandomForestRegressor(random_state=20)
rf.fit(X_train, y_train)
yhat=rf.predict(X_test)
print('Random', mean_absolute_error(y_test, yhat))

gb=GradientBoostingRegressor()
gb.fit(X_train, y_train)
yhat=rf.predict(X_test)
print('Gb', mean_absolute_error(y_test, yhat) )

import xgboost as xg
xgb_r = xg.XGBRegressor()
xgb_r.fit(X_train, y_train)
yhat=xgb_r.predict(X_test)
print('Xgb', mean_absolute_error(y_test, yhat) )

from catboost import CatBoostRegressor
ct= CatBoostRegressor(verbose=0)
ct.fit(X_train, y_train)
yhat=ct.predict(X_test)
print ('cat', mean_absolute_error(y_test, yhat) )

from sklearn.tree import DecisionTreeRegressor
dt=DecisionTreeRegressor()
dt.fit(X_train, y_train)
yhat=dt.predict(X_test)
print('dt', mean_absolute_error(y_test, yhat) )

Random 2191.679204074074
Gb 2191.679204074074
Xgb 2236.598237575955
cat 2133.283976316347
dt 2798.9466666666667


In [ ]:
# Save best model
from catboost import CatBoostRegressor
ct= CatBoostRegressor(verbose=0)
ct.fit(X, y)
ct.save_model('ct_abundance_global')

**Fungi Richness**

In [21]:
# Split to dependent and independent variables
X=df.drop(['abund', 'richness', 'plot'], axis=1)
y= df['richness']
# Split to train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.07, random_state=42)

In [ ]:
# Build regressor model for fungi richness prediction
rf=RandomForestRegressor(random_state=20)
rf.fit(X_train, y_train)
yhat=rf.predict(X_test)
print('Random', mean_absolute_error(y_test, yhat))

gb=GradientBoostingRegressor()
gb.fit(X_train, y_train)
yhat=rf.predict(X_test)
print('Gb', mean_absolute_error(y_test, yhat) )

import xgboost as xg
xgb_r = xg.XGBRegressor()
xgb_r.fit(X_train, y_train)
yhat=xgb_r.predict(X_test)
print('Xgb', mean_absolute_error(y_test, yhat) )

from catboost import CatBoostRegressor
ct= CatBoostRegressor(verbose=0)
ct.fit(X_train, y_train)
yhat=ct.predict(X_test)
print ('cat', mean_absolute_error(y_test, yhat) )

from sklearn.tree import DecisionTreeRegressor
dt=DecisionTreeRegressor()
dt.fit(X_train, y_train)
yhat=dt.predict(X_test)
print('dt', mean_absolute_error(y_test, yhat) )

Random 290.44989111111107
Gb 290.44989111111107
Xgb 296.7479515245226
cat 278.73311867737743
dt 381.61555555555555


In [ ]:
# Save best model
from catboost import CatBoostRegressor
ct= CatBoostRegressor(verbose=0)
ct.fit(X, y)
ct.save_model('ct_richness_global')

**Fungi trophic Mode diversity Prediction**

Clean dataset

In [ ]:
# path='/content/drive/MyDrive/global_fungi.guilds_filtered.csv.zip'
# import zipfile
# with zipfile.ZipFile(path, 'r') as zip_ref:
#     zip_ref.extractall()

In [22]:
import pandas as pd
df_guild=pd.read_csv('/content/global_fungi.guilds_filtered.csv')
df_guild.replace('Saportroph', 'Saprotroph', inplace=True)
df_guild.head()

,OTU,Kingdom,Phylum,Order,Class,Family,Genus,Species,taxon,taxonomicLevel,trophicMode,guild,trait,growthForm,confidenceRanking
0,850ed971e0e3fedba64b02b984b1c2af046d24fe,Fungi,Ascomycota,Leotiomycetes,Helotiales,Vibrisseaceae,Acephala,macrosclerotiorum,Acephala,13,Symbiotroph,Ectomycorrhizal,NaN,NaN,Possible
1,8bf5f5ba82723c4b98d6a667bae3465cbc07d445,Fungi,Ascomycota,Leotiomycetes,Helotiales,Vibrisseaceae,Acephala,macrosclerotiorum,Acephala,13,Symbiotroph,Ectomycorrhizal,NaN,NaN,Possible
2,b06ff698010bdfc59a9a7ae5c5e07d22960450f9,Fungi,Ascomycota,Leotiomycetes,Helotiales,Vibrisseaceae,Acephala,.,Acephala,13,Symbiotroph,Ectomycorrhizal,NaN,NaN,Possible
3,01cc0c41d82d43ab5b6ebd286a6d88c261dcda86,Fungi,Basidiomycota,Agaricomycetes,Russulales,Albatrellaceae,Albatrellus,.,Albatrellus,13,Symbiotroph,Ectomycorrhizal,NaN,Polyporoid,Highly Probable
4,0258ce9af3049f50c94d55c55e7c7fa6b85bc4c9,Fungi,Basidiomycota,Agaricomycetes,Russulales,Albatrellaceae,Albatrellus,.,Albatrellus,13,Symbiotroph,Ectomycorrhizal,NaN,Polyporoid,Highly Probable


In [23]:
df_guild.shape

(409646, 15)

In [24]:
trophic_modes=df_guild[['OTU', 'trophicMode']]

In [25]:
path='/content/drive/MyDrive/global_fungi_0.5.txt'
import pandas as pd
df=pd.read_table(path, on_bad_lines="skip", encoding='cp1252', delimiter="\t")

In [26]:
path='/content/drive/MyDrive/global_fungi_0.5.txt'
import pandas as pd
df=pd.read_table(path, on_bad_lines="skip", encoding='cp1252', delimiter="\t")
df_=df.T
df_.columns = df_.iloc[0]
df_.drop('OTU', inplace=True)
df_.reset_index()
df_
df_["Combination"] = df_.iloc[:, 1:].dot(df_.add_suffix("~").columns[1:]).str[:-2]
df_comb=df_["Combination"].str.split('~').to_frame()
df_comb=df_comb.reset_index()
df_explode=df_comb.explode('Combination')

In [27]:
merged=df_explode.merge(trophic_modes, how='left', left_on='Combination', right_on='OTU')
merged.fillna('Unassigned', inplace=True)

In [28]:
merged=merged.groupby('index')['trophicMode'].value_counts().to_frame()

In [29]:
merged.rename(columns={'trophicMode': 'Count'}, inplace=True)

In [30]:
merged.reset_index(inplace=True)

In [31]:
merged['Count']=merged['Count'].astype('str')

In [32]:
tro_count=merged.groupby(["index"]).agg (
   trophicMode=("trophicMode", ",".join), Count=("Count", ",".join))


In [33]:
tro_count['trophicMode']=tro_count['trophicMode'].str.split(',')

In [34]:
tro_count['Count']= tro_count['Count'].str.split(',')

In [35]:
big_dict=[]
for a, b in zip(tro_count.trophicMode.values, tro_count.Count.values):
  dict_={}
  for t, c in zip(a, b):
    dict_[t]=c
    big_dict.append(dict_)

In [36]:
final_=pd.DataFrame(big_dict).drop_duplicates()
final_.index= tro_count.index

In [37]:
final_.fillna(0, inplace=True)
final_=final_.astype('int64')

In [39]:
res = final_.div(final_.sum(axis=1), axis=0)

In [40]:
res=res.round(2)

In [41]:
df_=df_cleaned.merge(res, left_on='plot', right_index=True)

In [ ]:
X=df_.loc[:, 'Last_fire_y_ago': 'Biome specific']
X=X.drop(['abund','richness'], axis=1)

In [ ]:
Y=df_.loc[:, 'Unassigned': ]

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
for i in Y.columns:
  y=df_[i]
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.07, random_state=42)
  scaler=StandardScaler()
  X_train=scaler.fit_transform(X_train)
  X_test=scaler.transform(X_test)
  rf=RandomForestRegressor()
  rf.fit(X_train, y_train)
  y_pred=rf.predict(X_test)
  sc=mean_absolute_error(y_test, y_pred )
  print(i, sc, y.mean())

Unassigned 0.0769353925925926 0.18389375000000002
Saprotroph-Symbiotroph 0.07090992733686066 0.12119687500000001
Saprotroph 0.10760153439153439 0.225775
Pathotroph-Saprotroph-Symbiotroph 0.05172916507936508 0.08536250000000001
Symbiotroph 0.12774937248677248 0.292246875
Pathotroph-Symbiotroph 0.01140208888888889 0.012178125
Pathotroph 0.029419108994708995 0.0434875
Pathotroph-Saprotroph 0.029066319576719565 0.035296875000000005


In [ ]:
# Random forest
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
for i in Y.columns:
  y=df_[i]
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.07, random_state=42)
  scaler=StandardScaler()
  X_train=scaler.fit_transform(X_train)
  X_test=scaler.transform(X_test)
  gb= GradientBoostingRegressor()
  gb.fit(X_train, y_train)
  y_pred=gb.predict(X_test)
  sc=mean_absolute_error(y_test, y_pred )
  print(i, sc, y.mean())

Unassigned 0.07858510246206597 0.18389375000000002
Saprotroph-Symbiotroph 0.07187422254798985 0.12119687500000001
Saprotroph 0.10853260177474393 0.225775
Pathotroph-Saprotroph-Symbiotroph 0.05213636608326523 0.08536250000000001
Symbiotroph 0.13716629537503955 0.292246875
Pathotroph-Symbiotroph 0.01129495181512452 0.012178125
Pathotroph 0.02625540369647645 0.0434875
Pathotroph-Saprotroph 0.027902222863695113 0.035296875000000005


In [ ]:
# Xgboost
import xgboost as xg
xgb_r = xg.XGBRegressor()
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
for i in Y.columns:
  y=df_[i]
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.07, random_state=42)
  scaler=StandardScaler()
  X_train=scaler.fit_transform(X_train)
  X_test=scaler.transform(X_test)
  xgb_r.fit(X_train, y_train)
  y_pred=xgb_r.predict(X_test)
  sc=mean_absolute_error(y_test, y_pred )
  print(i, sc, y.mean())

Unassigned 0.08667284177243709 0.18389375000000002
Saprotroph-Symbiotroph 0.07520299370131558 0.12119687500000001
Saprotroph 0.11644413866003356 0.225775
Pathotroph-Saprotroph-Symbiotroph 0.05283373546800576 0.08536250000000001
Symbiotroph 0.14161558704359664 0.292246875
Pathotroph-Symbiotroph 0.01235621501615646 0.012178125
Pathotroph 0.029761233006014177 0.0434875
Pathotroph-Saprotroph 0.03343085992707767 0.035296875000000005


In [ ]:
# Catboost
from catboost import CatBoostRegressor
ct= CatBoostRegressor(verbose=0)

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
for i in Y.columns:
  y=df_[i]
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.07, random_state=42)
  ct.fit(X_train, y_train)
  y_pred=ct.predict(X_test)
  sc=mean_absolute_error(y_test, y_pred )
  print(i, sc, y.mean())

Unassigned 0.07675857959832552 0.18389375000000002
Saprotroph-Symbiotroph 0.07127730758257159 0.12119687500000001
Saprotroph 0.10659078986934978 0.225775
Pathotroph-Saprotroph-Symbiotroph 0.0499969259324971 0.08536250000000001
Symbiotroph 0.13674997262633415 0.292246875
Pathotroph-Symbiotroph 0.011196070654145466 0.012178125
Pathotroph 0.02647179593485019 0.0434875
Pathotroph-Saprotroph 0.028330752081523806 0.035296875000000005


In [ ]:
# Save model
for i in Y.columns:
  y=df_[i]
  ct.fit(X, y)
  ct.save_model(i)
